initialization and implementation of "get_log_prob_of_a_given_path()"

In [6]:
import math

In [ ]:
def get_log_prob_of_a_given_path(path, sequence):
    # Transition probabilities
    trans = {
        'E': {'E': math.log(0.9), '5': math.log(0.1), 'I': -math.inf, 'End': -math.inf},
        '5': {'E': -math.inf, '5': -math.inf, 'I': math.log(1.0), 'End': -math.inf},
        'I': {'E': -math.inf, '5': -math.inf, 'I': math.log(0.9), 'End': math.log(0.1)}
    }
    
    # Emission probabilities
    emit = {
        'E': {'A': math.log(0.25), 'C': math.log(0.25), 'G': math.log(0.25), 'T': math.log(0.25)},
        '5': {'A': math.log(0.05), 'C': -math.inf, 'G': math.log(0.95), 'T': -math.inf},
        'I': {'A': math.log(0.4), 'C': math.log(0.1), 'G': math.log(0.1), 'T': math.log(0.4)}
    }
    
    ans = 0.0
    
    # Initial state (Start -> First state in path)
    first_state=path[0]
    ans += math.log(1.0)  # Start -> E (forced by model)
    ans += emit[first_state][sequence[0]]  # First emission
    
    # Iterate through the rest of the path
    for i in range(1, len(path)):
        prev_state=path[i-1]
        curr_state=path[i]
        ans+=trans[prev_state][curr_state]  # Transition
        ans+=emit[curr_state][sequence[i]]  # Emission
    
    # Transition to End (if last state is 'I')
    if path[-1]== 'I':
        ans +=math.log(0.1)  # I -> End
    
    return ans

In [8]:
# Example from the primer
path = "EEEEEEEEEEEEEEEEEE5IIIIIII"
sequence ="CTTCATGTGAAAGCAGACGTAAGTCA"
print(get_log_prob_of_a_given_path(path, sequence))

-41.21967768602254


### Viterbi Algorithm

Say we're playing a guessing game with a friend who has a toy hidden in their hands. You can't see the toy, but your friend makes sounds that give you clues, like "meow" or "woof." You want to guess which toy it is – a cat or a dog.

The Viterbi algorithm is like a super smart guesser. It listens to all the sounds your friend makes and tries to figure out the best guess for the hidden toy at each sound. It remembers all the sounds and figures out the most likely way your friend was thinking ("Maybe first they thought of the cat, then the dog!"). 

Let P be all possible tags at each stage and L be the length of input sequence 

Complexity of brute force: O(p^L) --> exponential 

Complexity of Viterbi: O(L*p^2) -->quadratic --> huge improvement

Implementation using Iterative Dynamic Programming

In [ ]:
def viterbi(sequence):
    states = ['E', '5', 'I']
    emissions = ['A', 'C', 'G', 'T']
    
    # Transition probabilities (log space)
    trans = {
        'Start': {'E': math.log(1.0), '5': -math.inf, 'I': -math.inf, 'End': -math.inf},
        'E':{'E': math.log(0.9), '5': math.log(0.1), 'I': -math.inf, 'End': -math.inf},
        '5':{'E': -math.inf, '5': -math.inf, 'I':math.log(1.0), 'End': -math.inf},
        'I':{'E': -math.inf, '5': -math.inf, 'I': math.log(0.9), 'End':math.log(0.1)}
    }
    
    # Emission probabilities (log space)
    emit = {
        'E': {'A': math.log(0.25), 'C': math.log(0.25), 'G': math.log(0.25), 'T':math.log(0.25)},
        '5':{'A': math.log(0.05), 'C': -math.inf, 'G': math.log(0.95), 'T': -math.inf},
        'I': {'A': math.log(0.4), 'C': math.log(0.1), 'G': math.log(0.1), 'T': math.log(0.4)}
    }
    
    # Initialize Viterbi matrix and backpointer
    V = [{}]
    backpointer = [{}]
    
    # Initialization step (start in 'E')
    for s in states:
        V[0][s] = trans['Start'][s]+emit[s][sequence[0]] if s == 'E' else -math.inf
        backpointer[0][s] = 'Start'
    
    # Recursion step
    for t in range(1, len(sequence)):
        V.append({})
        backpointer.append({})
        for s in states:
            max_prob = -math.inf
            best_prev =None
            for prev_s in states:
                prob =V[t-1][prev_s]+ trans[prev_s][s] +emit[s][sequence[t]]
                if prob > max_prob:
                    max_prob = prob
                    best_prev = prev_s
            V[t][s] = max_prob
            backpointer[t][s] = best_prev
    
    # Termination step (transition to 'End')
    max_final = -math.inf
    best_final_state = None
    for s in states:
        prob = V[-1][s] +trans[s]['End']
        if prob >max_final:
            max_final= prob
            best_final_state= s
    
    # Backtracking
    best_path = []
    current_state = best_final_state
    best_path.append(current_state)
    for t in range(len(sequence)-1,0, -1):
        best_path.insert(0,backpointer[t][current_state])
        current_state= backpointer[t][current_state]
    
    return ''.join(best_path)

Testing on sequence given in primer 

In [10]:
sequence = "CTTCATGTGAAAGCAGACGTAAGTCA" 
print(viterbi(sequence))  # Output depends on emissions (e.g., "E5I" or "EEE")

EEEEEEEEEEEEEEEEEE5IIIIIII


## Reason for using log space: 
1. Probabilities shrink exponentially with sequence length and computers cannot represent extremely small numbers.

2. Log probabilities turn multiplication into addition (no underflow) which gives numerical         stability (adding log-probs is more stable than multiplying tiny floats).

3. Comparing log-probs is equivalent to comparing raw probabilities.